In [16]:
import pyodbc
import pandas as pd
import json

## 1. Connection à la base TestTech sur SQL Server

In [4]:
pyodbc.drivers()

['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server']

In [25]:
conn = pyodbc.connect(
    Trusted_Connected = "Yes",
    Driver = '{SQL Server}',
    Server = 'LAPTOP-IEBP4MPB\SQLEXPRESS',
    Database = "TestTech"
)

cursor = conn.cursor()

## 2. Pipeline de Data Ingestion  (ETL)

### Lecture des fichiers csv par pandas (Extract)

In [8]:
df_lieux = pd.read_csv('data/lieux.csv')
df_people = pd.read_csv('data/people.csv')

In [6]:
df_lieux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   commune      100 non-null    object
 1   departement  100 non-null    object
 2   region       100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [8]:
df_people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   prenom          10000 non-null  object
 1   nom             10000 non-null  object
 2   date-naissance  10000 non-null  object
 3   commune         10000 non-null  object
dtypes: object(4)
memory usage: 312.6+ KB


### Traitement des données (Transform)

In [9]:
df_people.rename(columns = {'date-naissance':'date_naissance'}, inplace = True)

In [10]:
df_people['date_naissance'] = pd.to_datetime(df_people.date_naissance)
df_people

,prenom,nom,date_naissance,commune
0,Camille,Collin,2021-07-31,Champigny-sur-Marne
1,Astrid,Fournier,1987-08-15,Sarcelles
2,Danielle,Vallée,1997-07-22,Montreuil
3,Sabine,Bailly,2002-06-07,Saint-Nazaire
4,Agathe,Guillon,1984-11-12,Paris
...,...,...,...,...
9995,Susanne,Jacob,1974-02-23,Aix-en-Provence
9996,Céline,Léger,1977-07-15,Bordeaux
9997,Marine,Alexandre,1981-03-21,Metz
9998,Maryse,Marion,1990-12-12,Issy-les-Moulineaux


### Chargement des données sur la base TestTech sur SQL Server (Load)

In [11]:
for row in df_lieux.itertuples():
    cursor.execute('''
                INSERT INTO lieux (commune, departement, region)
                VALUES (?,?,?)
                ''',
                row.commune, 
                row.departement,
                row.region
                )
for row in df_people.itertuples():
    cursor.execute('''
                INSERT INTO people (prenom, nom, date_naissance, commune)
                VALUES (?,?,?,?)
                ''',
                row.prenom, 
                row.nom,
                row.date_naissance,
                row.commune
                )
conn.commit()

## 3. Génération des résultats

In [14]:
cursor.execute('''
SELECT lieux.departement, COUNT(people.id_people) 
FROM people
INNER JOIN lieux ON lieux.commune = people.commune
GROUP BY lieux.departement
''')
departements_rows = cursor.fetchall()
departements_res = {}
for r in departements_rows:
    departements_res[r[0]] = r[1]

In [15]:
cursor.execute('''
SELECT lieux.region, COUNT(people.id_people) 
FROM people
INNER JOIN lieux ON lieux.commune = people.commune
GROUP BY lieux.region
''')
regions_rows = cursor.fetchall()
regions_res = {}
for r in regions_rows:
    regions_res[r[0]] = r[1]

In [26]:
cursor.close()

In [22]:
regions_json = "./results/regions.json"
data_region = json.dumps(regions_res, sort_keys=True, ensure_ascii=False)
with open(regions_json,"w",encoding="utf8") as rj:
    rj.write(data_region)


In [21]:
departements_json = "./results/departements.json"
data_departement = json.dumps(departements_res, sort_keys=True, ensure_ascii=False)
with open(departements_json,"w",encoding="utf8") as dj:
    dj.write(data_departement)